In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd

In [2]:
data = pd.DataFrame(columns=['year', 'speaker', 'speech', 'n_words'])

In [3]:
data

,year,speaker,speech,n_words


In [4]:
base_url = "https://hawzah.net"
url_77 = base_url + "/fa/Subjects/47895"
url_78 = base_url + "/fa/Subjects/47894"
url_79 = base_url + "/fa/Subjects/47893"
url_80 = base_url + "/fa/Subjects/47892"

In [5]:
def get_page_links(year_url):
    request = requests.get(year_url)
    response_content = request.content
    soup = BeautifulSoup(response_content, 'html.parser')
    pages_to_visit = []
    for link in soup.find_all('a'):
        link_str = link.get('href')
        if link_str and 'امامت' in link_str:
            pages_to_visit.append(base_url + link_str)
        
    return pages_to_visit

In [6]:
def get_text(pages, index, year):
    request = requests.get(pages[index])
    response_content = request.content
    soup = BeautifulSoup(response_content, 'html.parser')
    speech = []
    for p in soup.find_all('p'):
        if '<p class=' in str(p) and 'item-abstract hidden-xs' in str(p):
            break
        p = re.sub(r'<.+?>', '', str(p))
        p_lst = list(p)
        for i, char in enumerate(p_lst):
            if ord(char) > 2000 or ord(char) < 1000:
                p_lst[i] = ' '
        p = "".join(p_lst)
        words_lst = p.split()
        speech.extend(words_lst)
    n_words = len(speech)
    name = get_name(pages[index])
    
    return {'year': year, 'speaker': name, 'speech': speech, 'n_words': n_words}

In [7]:
def get_name(url):
    name = re.search('امامت-.+?-.+?-.+?-', url)
    if name:
        name = name.group(0).replace('-', '')
        name = name.replace('الله', '').replace('اله', '')
        name = name.replace('آیت', '').replace('امامت', '')
        return name
    return "NA"

In [8]:
# year 1377
pages = get_page_links(url_77)
for index, _ in enumerate(pages):
    row =  get_text(pages, index, '1387')
    data.loc[len(data.index)] = row

In [9]:
# year 1378
pages = get_page_links(url_78)
for index, _ in enumerate(pages):
    row =  get_text(pages, index, '1388')
    data.loc[len(data.index)] = row

In [10]:
# year 1379
pages = get_page_links(url_79)
for index, _ in enumerate(pages):
    row =  get_text(pages, index, '1389')
    data.loc[len(data.index)] = row

In [11]:
# year 1380
pages = get_page_links(url_80)
for index, _ in enumerate(pages):
    row =  get_text(pages, index, '1390')
    data.loc[len(data.index)] = row

In [12]:
data

,year,speaker,speech,n_words
0,1387,جنتی,"[حضور, پرشور, مردم, در, راهپیمایی, بهمن, درس, ...",710
1,1387,یزدی,"[خطبه, اول, بسم, الله, الرحمن, الرحیم, الحمد, ...",6265
2,1387,رفسنجانی23,"[خطبه, اول, بسم, الله, الرحمن, الرحیم, والصلاه...",6554
3,1387,جنتی,"[خطبه, اول, بسم, الله, الرحمن, الرحیم, والصلاه...",5334
4,1387,کاشانی,"[خطبه, اول, بسم, الله, الرحمن, الرحیم, نحمده, ...",5312
...,...,...,...,...
114,1390,یزدی,"[بسم, الله, الرحمن, الرحیم, الحمد, لله, رب, ال...",9197
115,1390,جنتی,"[بسم, الله, الرحمن, الرحیم, الحمد, لله, رب, ال...",4930
116,1390,هاشمی,"[بسم, الله, الرحمن, الرحیم, الحمدلله, رب, العا...",5267
117,1390,کاشانی,"[بسم, الله, الرحمن, الرحیم, الحمدالله, رب, الع...",3447


In [14]:
data.to_csv('../data/speeches_87to90.csv', sep='\t')